In [11]:
# =========================
# IMPORTS
# =========================

from X_functions import *
from X_config import *
from X_models import *
from X_interpolation import *
from X_plots import *

# =========================

In [2]:
# =========================

target_datetime=dt.datetime.now(dt.timezone.utc)

config = GGS_config_static(date=target_datetime)
root_directory = GGS_config_output(config, path="default")

datetime_index = [target_datetime.replace(hour=0, minute=0, second=0, microsecond=0).strftime('%Y-%m-%dT%H:%M:%SZ')]
datetime_index = datetime_index[0]
glider_data = None

sub_directory_plots = os.path.join(root_directory, "plots", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_plots, exist_ok=True)
sub_directory_data = os.path.join(root_directory, "data", ''.join(datetime_index[:10].split('-')))
os.makedirs(sub_directory_data, exist_ok=True)

# =========================

# model_datasets = []

# cmems = CMEMS(username='sfricano1', password='GlobalGliders1')
# cmems.cmems_load(config, datetime_index, subset_extent=True, subset_depth=True)
# cmems.cmems_save(config, sub_directory_data, save_data=False, save_qc=False)
# cmems_model_data = cmems.data

# cmems_depth_average, cmems_bin_average = interpolate_cmems(config, sub_directory_data, cmems_model_data, chunk=True, save_depth_average=True, save_bin_average=False)

# cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
# model_datasets.append(cmems_datasets)

# =========================

# latitude_qc = 0
# longitude_qc = 0

# GGS_plot_magnitudes(config, sub_directory_plots, datetime_index, model_datasets, latitude_qc=latitude_qc, longitude_qc=longitude_qc, density=2, gliders=glider_data, show_route=False, show_qc=False, show_eez=True, manual_extent=None)
# GGS_plot_threshold(config, sub_directory_plots, datetime_index, model_datasets, latitude_qc=latitude_qc, longitude_qc=longitude_qc, density=2, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, gliders=glider_data, show_route=False, show_qc=False, show_eez=True, manual_extent=None)
# GGS_plot_advantage(config, sub_directory_plots, datetime_index, model_datasets, latitude_qc=latitude_qc, longitude_qc=longitude_qc, density=2, tolerance=15, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, gliders=glider_data, show_route=True, show_qc=False, show_eez=True, manual_extent=None)
# GGS_plot_profiles(config, sub_directory_plots, datetime_index, model_datasets, latitude_qc=latitude_qc, longitude_qc=longitude_qc, threshold=0.5)

# =========================



### Glider Guidance System (GGS) Configuration ###

Glider name: SENTINEL

Execution date: 2024-03-21 00:17:23.940528+00:00

Max Depth: 1000

Extent:
  Boundary 1: (10, -78)
  Boundary 2: (44, -10)

GPS_coords:
  GPS_coord 1: (41.675, -70.522)
  GPS_coord 2: (15.067, -23.65)



---

In [ ]:
cmems_model_data = xr.open_dataset('C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/SENTINEL1_CMEMS_DepthAverage_20240321T00Z.nc')
cmems_depth_average = xr.open_dataset('C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/SENTINEL1_CMEMS_DepthAverage_20240321T00Z.nc')
cmems_bin_average = xr.open_dataset('C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/SENTINEL1_CMEMS_DepthAverage_20240321T00Z.nc')

cmems_datasets = (cmems_model_data, cmems_depth_average, cmems_bin_average)
model_datasets.append(cmems_datasets)

GGS_plot_advantage(config, sub_directory_plots, datetime_index, model_datasets, latitude_qc=latitude_qc, longitude_qc=longitude_qc, density=2, tolerance=15, mag1=0.0, mag2=0.2, mag3=0.3, mag4=0.4, mag5=0.5, gliders=glider_data, show_route=True, show_qc=False, show_eez=True, manual_extent=None)


---

In [ ]:
# import geopandas as gpd
# import simplekml
# import numpy as np
# import pandas as pd

# def dataset_to_kml_shp(config, directory, depth_average_data):
    
#     dataset_datetime = depth_average_data.attrs['model_datetime']
#     file_datetime = format_save_datetime(dataset_datetime)
    
#     depth_average_data = depth_average_data['mag_depth_avg'].isel(time=0)
    
#     lat, lon = np.meshgrid(depth_average_data['lat'].values, depth_average_data['lon'].values, indexing='ij')
#     mag = depth_average_data.values.flatten()
#     valid = ~np.isnan(mag)
    
#     depth_average_dataframe = pd.DataFrame({
#         'Longitude': lon.flatten()[valid],
#         'Latitude': lat.flatten()[valid],
#         'Magnitude': mag[valid]
#     })
    
#     gdf = gpd.GeoDataFrame(depth_average_dataframe, geometry=gpd.points_from_xy(depth_average_dataframe.Longitude, depth_average_dataframe.Latitude))
#     shp_filename = f"DepthAverageMagnitude_{config['max_depth']}m_{file_datetime}.shp"
#     shp_path = os.path.join(directory, shp_filename)
#     gdf.to_file(shp_path)
    
#     kml = simplekml.Kml()
#     for index, row in depth_average_dataframe.iterrows():
#         kml.newpoint(name=f"Magnitude: {row['Magnitude']}", coords=[(row['Longitude'], row['Latitude'])])
#     kml_filename = f"DepthAverageMagnitude_{config['max_depth']}m_{file_datetime}.kml"  # Corrected the extension here
#     kml_path = os.path.join(directory, kml_filename)
#     kml.save(kml_path)

# dataset_to_kml_shp(config, sub_directory_data, cmems_depth_average)

In [ ]:
# import xarray as xr
# import numpy as np
# import dask.array as da

# bathymetry_path = config["bathymetry_path"]
# bathy_data = xr.open_dataset(bathymetry_path, chunks={'lat': 500, 'lon': 500})

# # Define the function to create a mask for values close to the isobaths
# def mask_isobaths(elevation, levels=[-100, -1000], tolerance=10):
#     # Convert elevation to a dask array for manual operations
#     elevation_dask = elevation.data
#     mask = da.zeros(elevation_dask.shape, dtype=bool)
#     # Update the mask for each isobath level
#     for level in levels:
#         mask |= (elevation_dask >= level - tolerance) & (elevation_dask <= level + tolerance)
#     return xr.DataArray(mask, dims=elevation.dims, coords=elevation.coords)

# isobath_mask = mask_isobaths(bathy_data['elevation'])
# isobath_mask_computed = isobath_mask.compute()
# filtered_data = bathy_data.where(isobath_mask_computed, drop=True)

# # Compute the filtered data (if necessary and feasible)
# # Note: This step may still be memory-intensive depending on your dataset's size and system's capacity.
# # Consider further chunking or working with subsets if memory errors persist.
# # filtered_data = filtered_data.compute()

# # Save the filtered dataset to a new file
# path_to_save = 'C:/Users/sal_f/OneDrive/Desktop/STF-0/!-GGS/GGS_Scripts/data/isobaths_data.nc'
# filtered_data.to_netcdf(path_to_save)

# print(f'Saved filtered isobath data to {path_to_save}')
